# Table 4 / Table B7: Rotemberg Diagnosis (females under 20)

## Required Packages

- [`bartik_weight`](https://github.com/paulgp/bartik-weight)  
  - Please install the package following the instructions on the GitHub repository.

# Code

## Initializing

In [1]:
from IPython.display import HTML

HTML("""
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:110% !important; }</style>"))

In [3]:
# For mac-OS environment (Kobayahsi)
import stata_setup
stata_setup.config("/Applications/stata", "mp")

import pandas as pd
import numpy as numpy


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706353196
  Licensed to: Kobayashi Ryuki
               Keio University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [4]:
%%stata
qui{
capture restore    
use ../data/suicide_analysis_data_static,clear

xtset citycode covid  

/* Local macro */
local clsvars="citycode"

local gmm2="gmm2s"
local log_="log_"
sum suicide
local suicide_t_m=r(mean)
sum suicide_m
local suicide_m_m=r(mean)
sum suicide_f
local suicide_f_m=r(mean)
sum `log_'active_opening_ratio
local log_active_opening_ratio_m=r(mean)
sum `log_'unemployment_rate
local log_unemployment_rate_m=r(mean)
sum `log_'patient
local log_patient_m=r(mean)
sum log_active_opening_weighted
local log_active_opening_weighted_m=r(mean)
sum log_bankruptcy_debt
local log_bankruptcy_debt_m=r(mean)

* take first difference    
foreach x in u20 20_40 40_60 o60{
  gen D_suicide_f_`x'_log=D.suicide_f_`x'_log  
}
    
    
* shift share make
gen share_0=commuter_ratio_time00 
gen share_1=commuter_ratio_time30u
gen share_2=commuter_ratio_time30_60
gen share_3=commuter_ratio_time60_90
gen share_4=commuter_ratio_time90_120
gen share_5=commuter_ratio_time120o

gen shift_0=telework_ratio_time00_ac-telework_ratio_time00_bc    
gen shift_1=telework_ratio_time30u_ac-telework_ratio_time30u_bc    
gen shift_2=telework_ratio_time30_60_ac-telework_ratio_time30_60_bc
gen shift_3=telework_ratio_time60_90_ac-telework_ratio_time60_90_bc
gen shift_4=telework_ratio_time90_120_ac-telework_ratio_time90_120_bc
gen shift_5=telework_ratio_time120o_ac-telework_ratio_time120o_bc 

gen D_shift_share=0    
local variables_shift_share="" 
local variables_iv=""    
foreach x of numlist 0/5{
  gen iv_`x'=share_`x'*shift_`x'    
  replace D_shift_share=D_shift_share+iv_`x' 
  local variables_shift_share="`variables_shift_share' share_`x' shift_`x'  "     
  local variables_iv="`variables_iv' iv_`x'"    
}   
local variables_orders_all=""     
local variables_orders_all="`variables_shift_share' `variables_iv' D_shift_share" 

local sfx="u20"
sum suicide_f_`sfx'
local suicide_f_`sfx'_m=r(mean)
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_`sfx'_m')/(`suicide_f_`sfx'_m')) "
}


. qui{

. 


## Table 4: Panel B

In [5]:
%%stata
qui{
qui: set linesize 200

/* bartik */
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=D_commuting_time_iv), gmm2s first cluster(`clsvars')  
** Save 2nd stage result
nlcom `paracomp_f_stayhome', post 
est store suicide_f_2nd   
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=D_commuting_time_iv), gmm2s first cluster(`clsvars') 
weakivtest
estadd scalar F_eff=r(F_eff),replace : suicide_f_2nd
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_2nd
    
/* the largest rotemberg */
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time60_90), gmm2s first cluster(`clsvars')  savefirst savefprefix(F_1st_)
est store suicide_f_1st_60_90
** Save 2nd stage result
nlcom `paracomp_f_stayhome', post 
est store suicide_f_2nd_60_90     
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time60_90), gmm2s first cluster(`clsvars')  
weakivtest
estadd scalar F_eff=r(F_eff),replace : suicide_f_2nd_60_90
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_2nd_60_90

    
/* the second largest rotemberg */
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time30_60), gmm2s first cluster(`clsvars')  savefirst savefprefix(F_1st_)
est store suicide_f_1st_30_60
** Save 2nd stage result
nlcom `paracomp_f_stayhome', post 
est store suicide_f_2nd_30_60    
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time30_60), gmm2s first cluster(`clsvars')  
weakivtest
estadd scalar F_eff=r(F_eff),replace : suicide_f_2nd_30_60
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_2nd_30_60

/* the largest and second largest rotemberg */
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time30_60 commuter_ratio_time60_90), gmm2s  cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store suicide_f_1st_30_60_90
** Save 2nd stage result
nlcom `paracomp_f_stayhome', post 
est store suicide_f_2nd_30_60_90     
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time30_60 commuter_ratio_time60_90), gmm2s  cluster(`clsvars')  
local hj= e(j)
local hjp=e(jp)
estadd scalar hansenj= `hj' : suicide_f_2nd_30_60_90  
estadd scalar  hansenjp=`hjp'  : suicide_f_2nd_30_60_90       
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time30_60 commuter_ratio_time60_90), gmm2s  cluster(`clsvars') 
weakivtest    
estadd scalar F_eff=r(F_eff),replace : suicide_f_2nd_30_60_90
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_2nd_30_60_90

/* all ivs */
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time00 commuter_ratio_time30u commuter_ratio_time30_60 commuter_ratio_time60_90 commuter_ratio_time90_120 commuter_ratio_time120o), gmm2s  cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store suicide_f_1st_all
** Save 2nd stage result
nlcom `paracomp_f_stayhome', post 
est store suicide_f_2nd_all
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time00 commuter_ratio_time30u commuter_ratio_time30_60 commuter_ratio_time60_90 commuter_ratio_time90_120 commuter_ratio_time120o), gmm2s  cluster(`clsvars') 
local hj= e(j)
local hjp=e(jp)
estadd scalar hansenj= `hj',replace : suicide_f_2nd_all
estadd scalar hansenjp=`hjp',replace  : suicide_f_2nd_all     
ivreg2 D.suicide_f_`sfx'_log (D.stay_home=commuter_ratio_time00 commuter_ratio_time30u commuter_ratio_time30_60 commuter_ratio_time60_90 commuter_ratio_time90_120 commuter_ratio_time120o), gmm2s  cluster(`clsvars') 
weakivtest    
estadd scalar F_eff=r(F_eff),replace : suicide_f_2nd_all
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_2nd_all


#delimit;
noi: esttab 
suicide_f_2nd_60_90 
suicide_f_2nd_30_60
suicide_f_2nd_30_60_90
suicide_f_2nd_all    , 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats( F_eff c_TSLS_10  hansenj hansenjp N, layout(@ @ @ @ @ ) fmt( 3 3 3 3 %5.0f) labels( "Effective F statistic" "tau=10\%" "Hansen" "Hansen p-val" "Observation")) 
mgroups("60-90min"  "30-60min" "30-60min & 60-90min" "all inclusive", pattern(1 1 1 1)   )       
varwidth(20) modelwidth(20);  
};



. qui{

--------------------------------------------------------------------------------------------------------
                                 60-90min             30-60min  30-60min & 60-90min        all inclusive
                                      (1)                  (2)                  (3)                  (4)
                      D.suicide_f_u20_log  D.suicide_f_u20_log  D.suicide_f_u20_log  D.suicide_f_u20_log
                                   b/se/p               b/se/p               b/se/p               b/se/p
--------------------------------------------------------------------------------------------------------
StayHome                            6.820                2.415                5.173                4.505
                                  (2.246)              (2.570)              (2.040)              (2.002)
                                  [0.002]              [0.347]              [0.011]              [0.024]
----------------------------------------------

## Preparing Rotemberg

### original beta

In [6]:
%%stata
qui{
keep if covid==1       
local y D_suicide_f_`sfx'_log
local x D_stay_home
local z D_shift_share
order `y' `x' `z' `variables_orders_all'    
ivreg2 D_suicide_f_`sfx'_log (D_stay_home=D_shift_share), gmm2s cluster(`clsvars')
noi: di " beta"
noi: di _b[D_stay_home]*(1+`suicide_f_`sfx'_m')/(`suicide_f_`sfx'_m')
}


. qui{
 beta
4.8226842

. 


### first stage

In [7]:
%%stata
qui{
local share_stub share_
local shift_stub shift_
local iv_stub iv_

qui reg `x' `z'  , cluster(`clsvars')
scalar Fa = e(F)
noi: di "First Stage F value"
noi: di Fa
}


. qui{
First Stage F value
1964.1835

. 


In [8]:
%%stata
qui{
foreach var of varlist `share_stub'* {
    if regexm("`var'", "`share_stub'(.*)") {
    local k = regexs(1) 
    }
    tempvar temp
    qui gen `temp' = `var' * `shift_stub'`k'
    qui reg `x' `temp' , cluster(`clsvars')
    local pi_`k' = _b[`temp']
    qui test `temp'
    scalar Fv = e(F)
    local F_`k' = Fv

    qui reg `y' `temp' ,  cluster(`clsvars')
    local gamma_`k' = _b[`temp']
    drop `temp'
}
}                                      


. qui{

. 


In [9]:
%%stata
qui{
tempfile temp
preserve
foreach var of varlist `variables_iv' {
  egen var_`var' = sd(`var')
  replace var_`var' = var_`var'^2
}
keep if _n==1
keep var_*
gen id=_n
reshape long  var_iv_,i(id) j(cat)
drop id
save `temp',replace
restore
}


. qui{

. 


### Rotemberg

In [10]:
%%stata
qui{
drop if share_0==.
noi: bartik_weight, z(`share_stub'*)  weightstub(`shift_stub'*) x(`x') y(`y') 

mat beta = r(beta)*(1+`suicide_f_`sfx'_m')/(`suicide_f_`sfx'_m')
mat alpha = r(alpha)
mat gamma = r(gam)
mat pi = r(pi)
mat G = r(G)
qui desc `share_stub'*, varlist
local varlist = r(varlist)

svmat beta
svmat alpha
svmat gamma
svmat pi
svmat G  
save rotemberg_data,replace
}



. qui{
Controls: 
X variable is D_stay_home
Y variable is D_suicide_f_u20_log
No BY variable 

. 


In [11]:
%%stata
qui{
gen cat = ""
local t = 1
foreach var in `varlist' {
    if regexm("`var'", "`share_stub'(.*)") {
    qui replace cat = regexs(1) if _n == `t'
    }
    local t = `t' + 1
}
destring(cat),replace force
keep  cat beta1* alpha* gamma* pi* G*
drop if cat==.
gen commute_time="zero min"
replace commute_time="      less than 30 min" if cat==1
replace commute_time=" between 30 and 60 min" if cat==2
replace commute_time=" between 60 and 90 min" if cat==3
replace commute_time="between 90 and 120 min" if cat==4
replace commute_time="     more than 120 min" if cat==5
noi: list cat commute_time beta1 alpha1 G1,table clean  noobs  
}


. qui{

    cat             commute_time      beta1     alpha1         G1  
      0                 zero min   1.812932          0          0  
      1         less than 30 min   4.619551   -.397311   .0504364  
      2    between 30 and 60 min   2.414526   .5506502   .1550137  
      3    between 60 and 90 min   6.820117   .6590508   .1883896  
      4   between 90 and 120 min   4.451021   .1845576   .2121193  
      5        more than 120 min   4.013603   .0030525   .0420141  

. 


In [12]:
%%stata
qui{
gen alpha_sign=-1 if alpha1<0
replace  alpha_sign=0 if alpha1==0
replace  alpha_sign=1 if alpha1>0
label define sign -1 "negative (-)" 0 "0" 1 "positive (+)"
label value alpha_sign sign
noi tab  alpha_sign

gen F = .
gen pi = .
gen gamma = .
levelsof cat, local(cats)
foreach cat in `cats' {
	capture replace F = `F_`cat'' if cat == `cat'
	capture replace pi = `pi_`cat'' if cat == `cat'
	capture replace gamma = `gamma_`cat'' if cat == `cat'
	}
merge 1:1 cat using `temp'
noi: corr alpha1 G1 beta1 F var_iv
save rotemberg_result,replace
gen omega = alpha1*beta1
noi: total omega
mat b = e(b)
global omega = b[1,1]
noi: di $omega      
local b=$omega   
}



. qui{

  alpha_sign |      Freq.     Percent        Cum.
-------------+-----------------------------------
negative (-) |          1       16.67       16.67
           0 |          1       16.67       33.33
positive (+) |          4       66.67      100.00
-------------+-----------------------------------
       Total |          6      100.00
(obs=6)

             |   alpha1       G1    beta1        F  var_iv_
-------------+---------------------------------------------
      alpha1 |   1.0000
          G1 |   0.7015   1.0000
       beta1 |   0.2304   0.5085   1.0000
           F |   0.0575   0.3112   0.6799   1.0000
     var_iv_ |   0.7648   0.6081   0.3704   0.6093   1.0000


Total estimation                             Number of obs = 6

--------------------------------------------------------------
             |      Total   Std. err.     [95% conf. interval]
-------------+------------------------------------------------
       omega |   4.822685   5.154212      -8.42664    18.07

## Table B7: Panel A

In [13]:
%%stata
qui{
use rotemberg_result,clear
total alpha1 if  alpha_sign> 0
mat b = e(b)
local sum_pos_alpha = string(b[1,1], "%9.3f")
total alpha1 if  alpha_sign < 0
mat b = e(b)
local sum_neg_alpha = string(b[1,1], "%9.3f")
sum alpha1 if alpha_sign > 0
local mean_pos_alpha = string(r(mean), "%9.3f")
sum alpha1 if alpha_sign < 0
local mean_neg_alpha = string(r(mean), "%9.3f")
local share_pos_alpha = string(abs(`sum_pos_alpha')/(abs(`sum_pos_alpha') + abs(`sum_neg_alpha')), "%9.3f")
local share_neg_alpha = string(abs(`sum_neg_alpha')/(abs(`sum_pos_alpha') + abs(`sum_neg_alpha')), "%9.3f")
noi: di  "           Sum       Mean    Share "                                                                                
noi: di "Negative : `sum_neg_alpha'  `mean_neg_alpha'  `share_neg_alpha' " 
noi: di "Positive : `sum_pos_alpha'    `mean_pos_alpha'  `share_pos_alpha' "    
}                            


. qui{
           Sum       Mean    Share 
Negative : -0.397  -0.397  0.221 
Positive : 1.397    0.349  0.779 

. 


## Table B7: Panel B

In [14]:
%%stata
qui{
corr alpha1 G1 beta1 F var_iv
mat corr = r(C)
qui: mat list corr
forvalues i =1/5 {
forvalues j = `i'/5 {
local c_`i'_`j' = string(corr[`i',`j'], "%9.3g")
}
}
noi: di "correlation"
noi: di " alpha_k               1" 
noi: di " g_k                   `c_1_2'   1" 
noi: di " beta_k                `c_1_3'   `c_2_3'    1" 
noi: di " F_k                   `c_1_4'   `c_2_4'    `c_3_4'   1" 
noi: di " Var(z_k)              `c_1_5'   `c_2_5'    `c_3_5'   `c_4_5'   1" 
}


. qui{
correlation
 alpha_k               1
 g_k                   .702   1
 beta_k                .23   .508    1
 F_k                   .0575   .311    .68   1
 Var(z_k)              .765   .608    .37   .609   1

. 


## Table 4: Panel A / Table B7: Panel C

In [15]:
%%stata
qui{
use rotemberg_result,clear
 foreach cat in `cats' {
   sum alpha1 if cat == `cat'
   local alpha_`cat' = string(r(mean),"%9.3f")
   sum G1 if cat == `cat' 
   local G_`cat' = string(r(mean), "%9.3f")
   sum beta1 if cat == `cat'
   local beta_`cat' = string(r(mean),"%9.3f")  
   sum F if cat == `cat'
   local F_`cat' = string(r(mean),"%9.3f")      
   gen first_commute_time = commute_time   if cat == `cat'
   gsort - first_commute_time 
   list first_commute_time 
   local commute_time_`cat'=first_commute_time[1]
   drop first_commute_time    
}
use rotemberg_data,clear
foreach cat in `cats' {
   tempvar temp
   qui gen `temp' =  `share_stub'`cat' * `shift_stub'`cat'
   ch_weak, p(.05) beta_range(-10.0(.01)5.0)  y(`y') x(`x') z(`temp') cluster(`clsvars')
   local omega_min= r(beta_min)*(1+`suicide_f_`sfx'_m')/(`suicide_f_`sfx'_m')
   local omega_max= r(beta_max)*(1+`suicide_f_`sfx'_m')/(`suicide_f_`sfx'_m')
   local ci_min_`cat' =string(`omega_min', "%20.3f")
   local ci_max_`cat' =string(`omega_max', "%20.3f")
   if `cat'>=1{ 
   if `cat'==1{
   noi: disp  as text "     commuting time  : alpha     beta_k  F  G 95 % CI  "
   }
   noi: disp "`commute_time_`cat'':  `alpha_`cat''   `beta_`cat'' `F_`cat'' 　`G_`cat''   [`ci_min_`cat'', `ci_max_`cat'']"
   }
   drop `temp'
}
}


. qui{
     commuting time  : alpha     beta_k  F  G 95 % CI  
      less than 30 min:  -0.397   4.620 1807.293 　0.050   [0.626, 8.595]
 between 30 and 60 min:  0.551   2.415 722.220 　0.155   [-2.618, 7.456]
 between 60 and 90 min:  0.659   6.820 1681.096 　0.188   [2.448, 11.213]
between 90 and 120 min:  0.185   4.451 439.269 　0.212   [-1.651, 10.416]
     more than 120 min:  0.003   4.014 26.637 　0.042   [-12.010, 20.775]

. 


## Table B7: Panel D

In [16]:
%%stata
qui{
use rotemberg_result,clear
drop if alpha_sign==0    
gen positive_weight = alpha1 > 0
gen beta1_weight = beta1 * alpha1
collapse (sum) beta1_weight alpha1 (mean)  beta1, by(positive_weight)
gsort -positive_weight
gen beta1_share=beta1_weight/`b'    
local agg_beta_pos = string( beta1_weight[1], "%20.3f")
local agg_beta_neg = string( beta1_weight[2], "%20.3f")
local share_beta_pos = string( beta1_share[1], "%20.3f")
local share_beta_neg = string( beta1_share[2], "%20.3f")    
local agg_beta_pos2 = string(beta1[1], "%20.3f")
local agg_beta_neg2 = string(beta1[2], "%20.3f")
noi: di  as text " beta_k   : weight_sum    share sum "     
noi: di  as text " Negative : `agg_beta_neg'   `share_beta_neg'       `agg_beta_neg2' " 
noi: di  as text " Positive : `agg_beta_pos'   `share_beta_pos'       `agg_beta_pos2' " 
}


. qui{
 beta_k   : weight_sum    share sum 
 Negative : -1.835   -0.381       4.620 
 Positive : 6.658   1.381       4.425 

. 
